In [12]:
import os
import pandas as pd

In [13]:
all_image_files = os.listdir('data/images')
len(all_image_files)

8091

In [14]:
captions_vi = open('data/captions_vi.txt', 'r', encoding='utf8')
file_names, nums, caps = [], [], []

for caption in captions_vi:
    list_text = caption.strip('\n').split()
    if len(list_text) >= 4:
        if list_text[1] == '#':
            file_names.append(list_text[0])
            nums.append(list_text[1] + list_text[2])
            caps.append(' '.join(list_text[3:]))
    else:
        # print(caption)
        pass
        
df = pd.DataFrame({
    'file_name': file_names,
    'nums': nums,
    'caps': caps
})

df.shape

(40339, 3)

In [15]:
df.nunique()

file_name     8091
nums             5
caps         39826
dtype: int64

In [8]:
8091 * 5

40455

In [17]:
df.head()

,file_name,nums,caps
0,1000268201_693b08cb0e.jpg,#0,Một đứa trẻ mặc váy hồng đang trèo lên một bộ ...
1,1000268201_693b08cb0e.jpg,#1,Một cô gái đi vào một tòa nhà bằng gỗ.
2,1000268201_693b08cb0e.jpg,#2,Một cô bé trèo vào nhà chơi bằng gỗ.
3,1000268201_693b08cb0e.jpg,#3,Một cô bé leo lên cầu thang đến nhà chơi của m...
4,1000268201_693b08cb0e.jpg,#4,Một cô bé mặc váy hồng đi vào cabin gỗ.


In [24]:
# Kiểm tra có bị duplicate hay không
assert len(df.drop_duplicates()) == df.shape[0]

In [83]:
# Lọc ra tên các file ảnh có caption
dist_file = [file for file in all_image_files if file in df['file_name'].values.tolist()]
file_no_cap = set(all_image_files) - set(dist_file)
len(dist_file)

7938

Chỉ có 7938 ảnh trong số 8091 ảnh có sẵn caption

In [85]:
len(file_no_cap)

153

In [90]:
# Lấy ra những caption có ảnh
dist_df = df.loc[df.file_name.isin(dist_file)]
dist_df.shape

(39616, 3)

In [68]:
dist_df.head()

,file_name,nums,caps
0,1000268201_693b08cb0e.jpg,#0,Một đứa trẻ mặc váy hồng đang trèo lên một bộ ...
1,1000268201_693b08cb0e.jpg,#1,Một cô gái đi vào một tòa nhà bằng gỗ.
2,1000268201_693b08cb0e.jpg,#2,Một cô bé trèo vào nhà chơi bằng gỗ.
3,1000268201_693b08cb0e.jpg,#3,Một cô bé leo lên cầu thang đến nhà chơi của m...
4,1000268201_693b08cb0e.jpg,#4,Một cô bé mặc váy hồng đi vào cabin gỗ.


In [69]:
assert len(dist_file) * 5 == dist_df.shape[0], 'Không đủ 5 caption cho 1 ảnh'

AssertionError: Không đủ 5 caption cho 1 ảnh

In [70]:
# Lấy ra tên những file không có đủ 5 caption
file_missing_cap = []
for file_name in dist_file:
    if len(dist_df.loc[dist_df.file_name == file_name]) != 5:
        file_missing_cap.append(file_name)

In [71]:
len(file_missing_cap)

49

In [72]:
idx2drop = dist_df.loc[dist_df.file_name.isin(file_missing_cap)].index

In [91]:
# Dữ liệu có đủ ảnh và 5 caption
dist_df = dist_df.drop(idx2drop, axis=0)
dist_df.shape

(39445, 3)

In [184]:
file_to_trans = list(file_no_cap)[:63]

In [185]:
len(file_to_trans)

63

In [186]:
en_cap = pd.read_csv('data/captions.txt')
en_cap.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [187]:
from googletrans import Translator
import time
def dich(x):
    translator = Translator()
    translator= translator.translate(x,dest='vi')
    return translator.text

In [251]:
def rm_punct(text):
    my_punct = '\r[!"\#\$%\&\(\)\*\+,/:;<=>\?@\[\\\]\^_—`\{\|\}\~»«“”]\''
    trans = str.maketrans('', '', my_punct)
    return text.translate(trans)

In [189]:
tmp_df = pd.DataFrame(columns=['filename', 'captions'])

for i in file_to_trans:
    texts = en_cap.loc[en_cap['image']==i, 'caption'].values
    texts_out = [rm_punct(i).strip() for i in texts]
    b = dich(texts)
    d = {'filename':[i],'captions':b}
    df_2= pd.DataFrame(data=d)
    print(i + ': oke')
    tmp_df = pd.concat([tmp_df,df_2])
    time.sleep(3)

463978865_c87c6ca84c.jpg: oke
3457604528_302396c08c.jpg: oke
3132832452_c354c6396c.jpg: oke
1260816604_570fc35836.jpg: oke
2467803152_70eeca1334.jpg: oke
3070485870_eab1a75c6f.jpg: oke
314940358_ec1958dc1d.jpg: oke
2972929655_04233b5489.jpg: oke
2522230304_1581d52961.jpg: oke
3400135828_0ac128b6eb.jpg: oke
3189002057_3ef61b803e.jpg: oke
2754898893_95239c1f19.jpg: oke
1659396176_ced00a549f.jpg: oke
2303016989_0deb96c8d9.jpg: oke
2913972180_547783dd3d.jpg: oke
3000722396_1ae2e976c2.jpg: oke
1347519824_e402241e4f.jpg: oke
2684489465_32ba1d0344.jpg: oke
3430100177_5864bf1e73.jpg: oke
157139628_5dc483e2e4.jpg: oke
2410562803_56ec09f41c.jpg: oke
3379839396_0cd84b55f1.jpg: oke
3576312396_799c873f3e.jpg: oke
2700147489_f1664f2b61.jpg: oke
3547524138_4157f660b0.jpg: oke
3215315009_47577bf8f7.jpg: oke
218342358_1755a9cce1.jpg: oke
3330935489_9cb67ca36b.jpg: oke
3118505332_b0792489b5.jpg: oke
2559114800_17310f3015.jpg: oke
2752923489_d3bceebacb.jpg: oke
3712742641_641282803e.jpg: oke
534875358_6e

In [190]:
tmp_df.reset_index(drop=True, inplace=True)

In [199]:
tmp_df.to_csv('data/tmp_df.txt', index=False, header=True)

In [130]:
missing = pd.read_csv('data/abcde.csv')
missing.shape

(49, 2)

In [133]:
def tao_df(file_missing_cap, df):
    final = pd.DataFrame(
        columns=['filename', 'nums', 'captions']
    )
    
    num = ['#' + str(i) for i in range(5)]
    for idx, row in enumerate(file_missing_cap):
        tmp = pd.DataFrame({
            'filename': [row] * 5,
            'nums': num,
            'captions': rm_punct(df.loc[idx, 'captions']).split('\n')
        })
        final = pd.concat([final, tmp])
    final.reset_index(drop=True, inplace=True)
    final['captions'] = final['captions'].apply(rm_punct)
    return final

In [138]:
tao_df(file_missing_cap, missing)

,filename,nums,captions
0,1332722096_1e3de8ae70.jpg,#0,Ba người phụ nữ đứng trên một con phố trong th...
1,1332722096_1e3de8ae70.jpg,#1,Ba người phụ nữ đi bộ xuống phố.
2,1332722096_1e3de8ae70.jpg,#2,Hai cô gái và một người phụ nữ đi bộ trên vỉa hè
3,1332722096_1e3de8ae70.jpg,#3,Hai cô gái đi ủng màu xanh lá cây tươi sáng v...
4,1332722096_1e3de8ae70.jpg,#4,Hai cô gái trẻ đi ủng màu xanh lá cây đang đi...
...,...,...,...
240,900144365_03cd1899e3.jpg,#0,Một cô gái mặc đồ đen dẫn đầu trong một cuộc đ...
241,900144365_03cd1899e3.jpg,#1,Các vận động viên chạy marathon đang chạy đua...
242,900144365_03cd1899e3.jpg,#2,Người chạy trong cuộc đua chạy trên một con p...
243,900144365_03cd1899e3.jpg,#3,Những vận động viên đang thi đấu chạy nước rú...


In [208]:
tmp_df = pd.read_csv('data/tmp_df.txt')

In [205]:
tmp_df['counts'] = tmp_df['captions'].apply(lambda x: len(x.split('\n')))
tmp_df

,filename,captions,counts
0,463978865_c87c6ca84c.jpg,['Một người đàn ông ngồi bên trái gần một ngườ...,5
1,3457604528_302396c08c.jpg,"['Một cô bé mặc áo đỏ thả diều. ""\r\n 'Một cô ...",5
2,3132832452_c354c6396c.jpg,"[""Một ​​gia đình vây quanh đứa trẻ trong lòng ...",5
3,1260816604_570fc35836.jpg,"['Một người lái một chiếc xe hơi màu đỏ. ""\r\n...",5
4,2467803152_70eeca1334.jpg,['Một con chó lớn màu nâu chạy với một cây gậy...,5
...,...,...,...
58,3214579977_fa9fb006a6.jpg,"['Một con chó đen nhảy trong tuyết. ""\r\n 'Một...",5
59,3039675864_0b7961844d.jpg,"['Một người đàn ông đi xe máy xuống vỉa hè. ""\...",5
60,2284894733_b710b9b106.jpg,['Một con chó nâu gầm gừ.'\r\n 'Một con chó nâ...,5
61,2839789830_89668775a4.jpg,['Một con chó đen và trắng bắt Frisbee trên mộ...,5


In [209]:
final = pd.concat([tao_df(file_missing_cap, missing), tao_df(file_to_trans, tmp_df)])

In [219]:
final.columns = ['file_name', 'nums', 'caps']

In [222]:
data = pd.concat([final, dist_df])

In [229]:
data.reset_index(drop=True, inplace=True)

In [236]:
data = data.loc[:39999]

In [244]:
data.to_csv('data/new_captions_vi.csv', index=False, header=True, encoding='utf-8-sig')

In [245]:
data

,file_name,nums,caps
0,1332722096_1e3de8ae70.jpg,#0,Ba người phụ nữ đứng trên một con phố trong th...
1,1332722096_1e3de8ae70.jpg,#1,Ba người phụ nữ đi bộ xuống phố\r
2,1332722096_1e3de8ae70.jpg,#2,Hai cô gái và một người phụ nữ đi bộ trên vỉa...
3,1332722096_1e3de8ae70.jpg,#3,Hai cô gái đi ủng màu xanh lá cây tươi sáng v...
4,1332722096_1e3de8ae70.jpg,#4,Hai cô gái trẻ đi ủng màu xanh lá cây đang đi...
...,...,...,...
39995,997338199_7343367d7f.jpg,#0,Một người đứng gần những bức tường vàng.
39996,997338199_7343367d7f.jpg,#1,một phụ nữ đằng sau bức tường cuộn đang viết
39997,997338199_7343367d7f.jpg,#2,Một người phụ nữ đứng gần bức tường trang trí ...
39998,997338199_7343367d7f.jpg,#3,Các bức tường được phủ vàng và hoa văn.


In [246]:
conmeno = pd.read_csv('data/new_captions_vi.csv')
conmeno.head()

,file_name,nums,caps
0,1332722096_1e3de8ae70.jpg,#0,Ba người phụ nữ đứng trên một con phố trong th...
1,1332722096_1e3de8ae70.jpg,#1,Ba người phụ nữ đi bộ xuống phố\r
2,1332722096_1e3de8ae70.jpg,#2,Hai cô gái và một người phụ nữ đi bộ trên vỉa...
3,1332722096_1e3de8ae70.jpg,#3,Hai cô gái đi ủng màu xanh lá cây tươi sáng v...
4,1332722096_1e3de8ae70.jpg,#4,Hai cô gái trẻ đi ủng màu xanh lá cây đang đi...


In [255]:
conmeno['caps'] = conmeno['caps'].apply(rm_punct)

In [256]:
conmeno.to_csv('data/new_captions_vi.', index=False, header=True, encoding='utf-8-sig')

In [257]:
conmeno.tail()

,file_name,nums,caps
39995,997338199_7343367d7f.jpg,#0,Một người đứng gần những bức tường vàng.
39996,997338199_7343367d7f.jpg,#1,một phụ nữ đằng sau bức tường cuộn đang viết
39997,997338199_7343367d7f.jpg,#2,Một người phụ nữ đứng gần bức tường trang trí ...
39998,997338199_7343367d7f.jpg,#3,Các bức tường được phủ vàng và hoa văn.
39999,997338199_7343367d7f.jpg,#4,Người phụ nữ viết trên một tấm đệm trong phòng...
